In [142]:
import requests
import os
from dotenv import load_dotenv

load_dotenv(override=True)

True

In [144]:
API_KEY = os.environ.get("API_KEY")

In [130]:
bibleVersionID = {
    "ASV": "06125adad2d5898a-01",
    "WEB": "72f4e6dc683324df-01",
    "FBV": "65eec8e0b60e656b-01",
    "WMB": "f72b840c855f362c-04"
}

books_res = requests.get(
    url=f"https://api.scripture.api.bible/v1/bibles/{bibleVersionID['WMB']}/books",
    headers={
        "api-key": API_KEY,
        "Accept-Encoding": "application/json",
    },
)

In [131]:
print(books_res.json())

{'statusCode': 403, 'error': 'Forbidden', 'message': 'You are not authorized to access that bible or bibles'}


In [122]:
books_json = books_res.json()
books = []
for book in books_json["data"]:
    books.append(book["id"])

In [123]:
books[:5]

['GEN', 'EXO', 'LEV', 'NUM', 'DEU']

In [124]:
chapters = []
for book in books:
    chapters_res = requests.get(
        url=f"https://api.scripture.api.bible/v1/bibles/{bibleVersionID['WMB']}/books/{book}/chapters",
        headers={
            "api-key": API_KEY,
            "Accept-Encoding": "application/json",
        },
    )

    for chapter in chapters_res.json()["data"]:
        chapters.append(chapter["id"])

In [40]:
# verse_ids = []
# for chapter in chapters:
#     verses_res = requests.get(
#         url=f"https://api.scripture.api.bible/v1/bibles/{bibleVersionID}/chapters/{chapter}/verses",
#         headers={
#             "api-key": API_KEY,
#             "Accept-Encoding": "application/json",
#         },
#     )

#     for verse in verses_res.json()["data"]:
#         verse_ids.append(verse["id"])

In [52]:
# verse_ids_no_intro = filter(lambda x: "intro" not in x, verse_ids)
# len(list(verse_ids_no_intro))

In [125]:
chapters_no_intro = filter(lambda x: "intro" not in x, chapters)
chapters_no_intro = list(chapters_no_intro)

In [126]:
chapters_no_intro[:10], len(chapters_no_intro)

(['GEN.1',
  'GEN.2',
  'GEN.3',
  'GEN.4',
  'GEN.5',
  'GEN.6',
  'GEN.7',
  'GEN.8',
  'GEN.9',
  'GEN.10'],
 1189)

In [94]:
import pandas

In [146]:
for idx, chapter in enumerate(chapters_no_intro[822:]):
    chapter_verses_res = requests.get(
        url=f"https://api.scripture.api.bible/v1/bibles/{bibleVersionID['WMB']}/chapters/{chapter}",
        headers={
            "api-key": API_KEY,
            "Accept-Encoding": "application/json",
        },
        params={
            "content-type": "text",
            "include-notes": "false",
            "include-titles": "false",
        },
    )

    df = pandas.DataFrame(
        {
            "index": idx+822,
            "chapter": chapter,
            "verses": chapter_verses_res.json()["data"]["content"],
        },
        index=[0],
    )
    df.to_csv("data/wmb/wmb.csv", mode="a", header=False, index=False)

In [42]:
import re

In [147]:
df = pandas.read_csv("data/wmb/wmb.csv", index_col=0)

In [148]:
chapter_verses = []
for idx, verse in enumerate(df["verses"]):
    verse_clean = re.split(r"\s*\[\d+\]\s*", verse.strip())
    verse_clean = [item for item in verse_clean if item]
    for idx_verse, verse in enumerate(verse_clean):
        chapter_verses.append(
            {"citation": f"{df['chapter'][idx]}.{idx_verse+1}", "verse": verse}
        )

In [149]:
df_verses = pandas.DataFrame(chapter_verses)
df_verses.to_csv("data/wmb/wmb_clean.csv", index=False)

In [150]:
df_verses

,citation,verse
0,GEN.1.1,"In the beginning, God created the heavens and ..."
1,GEN.1.2,The earth was formless and empty. Darkness was...
2,GEN.1.3,"God said, “Let there be light,” and there was ..."
3,GEN.1.4,"God saw the light, and saw that it was good. G..."
4,GEN.1.5,"God called the light “day”, and the darkness h..."
...,...,...
31093,REV.22.17,"The Spirit and the bride say, “Come!” He who h..."
31094,REV.22.18,I testify to everyone who hears the words of t...
31095,REV.22.19,If anyone takes away from the words of the boo...
31096,REV.22.20,"He who testifies these things says, “Yes, I am..."


In [157]:
web = pandas.read_csv("data/web/web_clean_final.csv")
asv = pandas.read_csv("data/asv/asv_clean_final.csv")
kjv = pandas.read_csv("data/kjv/kjv_clean_final.csv")
fbv = pandas.read_csv("data/fbv/fbv_clean_final.csv")
wmb = pandas.read_csv("data/wmb/wmb_clean_final.csv")

In [158]:
len(web), len(asv), len(kjv), len(fbv), len(wmb)

(37499, 31087, 31102, 31088, 31098)